In [1]:
from data_collector import DataCollector
from data_preprocessor import DataPreprocessor
from model_handler import ModelHandler
from utils import clear_and_show_title, ensure_folders_exist
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import os

In [2]:
# If in src folder, go back to root folder
if os.getcwd().split('\\')[-1] == 'src':
    os.chdir('..')
print(os.getcwd())
asset_type = "stock"
placeholder_symbol = "MAT"
validation_size = 0.2
train_days = 90

c:\Users\callu\Desktop\The Lab\Brainchildren\crypto-analyst


In [5]:
import pandas as pd

# Create a Pandas DataFrame
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
print(df.head())
# Slice the DataFrame
sliced_df = df.iloc[1:, :2]
print()
print(sliced_df)

   A  B  C
0  1  4  7
1  2  5  8
2  3  6  9

   A  B
1  2  5
2  3  6
